In [1]:
import pandas as pd
import numpy as np
import os

import plotly.graph_objs as go

In [2]:
train_df = pd.read_csv('../train.csv')

In [3]:
train_df.head(3)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0


### Looking at the number of duplicates (using plotly 3.0!)

In [4]:
dup_df = train_df.\
         groupby('is_duplicate',
                 as_index = False)\
         ['id'].\
         count()


In [32]:
fig = go.FigureWidget(
    data = [go.Bar(
        x=dup_df['is_duplicate'],
        y=dup_df['id']
    )],
    layout=go.Layout(
        title="Number of records for each 'is_duplicate' class",
        titlefont=dict(
            color='rgb(230,230,230)'
        ),
        xaxis=dict(
            title='is duplicate',
            color='rgb(230,230,230)'
        ),
        yaxis=dict(
            title='Count',
            color='rgb(230,230,230)'
        ),
        paper_bgcolor = 'rgb(44,48,60)',
        plot_bgcolor = 'rgb(44,48,60)'
    )
)


In [33]:
fig

FigureWidget({
    'data': [{'type': 'bar',
              'uid': 'fddf6186-8106-11e8-8fb9-bc8385288ae2',
              'x': array([0., 1.]),
              'y': array([255027., 149263.])}],
    'layout': {'paper_bgcolor': 'rgb(44,48,60)',
               'plot_bgcolor': 'rgb(44,48,60)',
               'title': "Number of records for each 'is_duplicate' class",
               'titlefont': {'color': 'rgb(230,230,230)'},
               'xaxis': {'color': 'rgb(230,230,230)', 'title': 'is duplicate'},
               'yaxis': {'color': 'rgb(230,230,230)', 'title': 'Count'}}
})

### Let's take a look at how many times each question appears

In [7]:
train_df.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')

In [8]:
questions = pd.concat([
                train_df['qid1'],
                train_df['qid2'].\
                    rename(columns={'qid2' : 'qid1'})
                ],
                axis=0
)

In [9]:
questions = pd.DataFrame(questions).\
    rename(columns={0:'qid'})
# adding dummy indicator for aggregation
questions['ind'] = 1

In [10]:
questions = questions.\
            groupby('qid',
                     as_index = False)\
            ['ind'].\
            count().\
            sort_values('ind',
                         ascending=False
            ).\
            rename(columns={'ind':'count'})

In [11]:
questions.head(3)

,qid,count
2558,2559,157
30781,30782,120
4043,4044,111


In [34]:
# set this parameter to be whatever you want.
num_questions = 50

q_fig = go.FigureWidget(
    data = [go.Bar(
        x=questions['qid'].\
          apply(lambda x: 'qid: %i' % x).\
          head(num_questions),
        y=questions['count'].\
          head(num_questions)
    )],
    layout=go.Layout(
        title="Number of questions appearing in the training set for each qid (top %i questions)" % num_questions,
        titlefont=dict(
            color='rgb(230,230,230)'
        ),
        xaxis=dict(
            title='qid',
            color='rgb(230,230,230)'
        ),
        yaxis=dict(
            title='Count',
            color='rgb(230,230,230)'
        ),
        paper_bgcolor = 'rgb(44,48,60)',
        plot_bgcolor = 'rgb(44,48,60)'
    )
)


In [35]:
q_fig

FigureWidget({
    'data': [{'type': 'bar',
              'uid': '30dcd8f4-8107-11e8-ba25-bc8385288ae2',
              'x': array(['qid: 2559', 'qid: 30782', 'qid: 4044', 'qid: 2561', 'qid: 14376',
                          'qid: 17978', 'qid: 2675', 'qid: 28764', 'qid: 1772', 'qid: 24555',
                          'qid: 18753', 'qid: 4018', 'qid: 28133', 'qid: 3595', 'qid: 18531',
                          'qid: 19621', 'qid: 4951', 'qid: 38', 'qid: 13748', 'qid: 10024',
                          'qid: 10330', 'qid: 18296', 'qid: 11264', 'qid: 20190', 'qid: 6551',
                          'qid: 33412', 'qid: 3938', 'qid: 6749', 'qid: 691', 'qid: 2322',
                          'qid: 8461', 'qid: 12639', 'qid: 58173', 'qid: 4367', 'qid: 1885',
                          'qid: 11588', 'qid: 10382', 'qid: 2439', 'qid: 1749', 'qid: 11397',
                          'qid: 65693', 'qid: 27618', 'qid: 25984', 'qid: 7801', 'qid: 21693',
                          'qid: 17761', 'qid: 2921', 'qid: 6100', 'qid: 14110', 'qid: 17854'],
                         dtype=object),
              'y': array([157., 120., 111.,  88.,  79.,  77.,  68.,  66.,  63.,  61.,  58.,  57.,
                           56.,  55.,  55.,  55.,  55.,  54.,  54.,  54.,  54.,  53.,  52.,  52.,
                           52.,  51.,  51.,  51.,  51.,  51.,  51.,  51.,  50.,  50.,  50.,  50.,
                           50.,  49.,  49.,  49.,  49.,  49.,  49.,  49.,  49.,  49.,  48.,  48.,
                           48.,  47.])}],
    'layout': {'paper_bgcolor': 'rgb(44,48,60)',
               'plot_bgcolor': 'rgb(44,48,60)',
               'title': ('Number of questions appearing ' ... 'or each qid (top 50 questions)'),
               'titlefont': {'color': 'rgb(230,230,230)'},
               'xaxis': {'color': 'rgb(230,230,230)', 'title': 'qid'},
               'yaxis': {'color': 'rgb(230,230,230)', 'title': 'Count'}}
})